## Do Small Earthquake Leads to Bigger Earthquake?
**I will modify the question a little bit, I will try to find clusters of sequence of earthquakes
within short distance (approximately 10 km) and time (approximately 7 days).**

In [ ]:
#Importing necessary libaries

import pandas as pd
from sklearn.cluster import DBSCAN


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



from sklearn.cluster import DBSCAN
import math

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
# Reading data

df = pd.read_csv('../input/indian-earthquakes-dataset2018-onwards/Indian_earthquake_data.csv')
df.head()

In [ ]:
#PREPROCESSING
df['Origin Time'] = pd.to_datetime(df['Origin Time'])

In [ ]:
#FIND NULL VALUES
df.isna().sum()

# STATISTICAL ANALYSIS AND VISUALIZATION

In [ ]:
# mean ,and Quantile of Magnitude
q1 = df["Magnitude"].quantile(.25)
q2 = df["Magnitude"].quantile(.5)
q3 = df["Magnitude"].quantile(.75)

q90 = df["Magnitude"].quantile(.9)
q95 = df["Magnitude"].quantile(.95)
q99 = df["Magnitude"].quantile(.99)

mean =df["Magnitude"].mean() 
print("mean = ",mean,"\nFirst Quartile = ",q1)
print("Second Quartile = ",q2,"\nThird Quartile = ",q3)


print("99th Quantile = ",q99)

## Clustering based on nearest earthquake
IN THIS SECTION I WILL TRY TO CLUSTER NEAREST EARTHQUAKES 
* I will try to find earthquakes within 10 km
* We are using DBSCAN. All the points in the cluster may not have the same inter-distance within the limit (10km)  .Eg, the distance between the first point and the last point may be greater than 10 km. 
* Also for the time clustering ,all the points in the cluster may not have the same time interval within the limit (7 days).Eg, the time distance between the first point and the last point may be greater than 7 days. 

In [ ]:

def cluster_distance(lat,lon):

    # custom distance funciton for clustering
    # returns distance betowin points on the earth surface in km
    
    
    lat1,lon1= lat 
    lat2,lon2= lon
    
    
    R = 6371e3; # earth 
    f1 = lat1 * math.pi/180; # phi lambda in radians
    f2 = lat2 * math.pi/180;
    delf = (lat2-lat1) * math.pi/180;
    dellambda = (lon2-lon1) * math.pi/180;

    a = math.sin(delf/2) * math.sin(delf/2) +  math.cos(f1) * math.cos(f2) *    math.sin(dellambda/2) * math.sin(dellambda/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a));

    d = R * c; # in metres

    return d/1000 # in km



 

X = df[['Latitude', 'Longitude']].values 


aa = DBSCAN( eps=5,min_samples=1,metric=cluster_distance)
aa = aa.fit(X)

In [ ]:
# setting lables for clusters 
df["cluster_label"] = aa.labels_

In [ ]:
print("NUMBER OF CLUSTERS ",aa.labels_.max()) 

### Clustergin based on time for each nearest earth quake

In [ ]:
earth_quake_with_sequence = 0
earth_quake_without_sequence = 0

def cluster_distance_time(time1,time2):
    diff = pd.to_datetime(time1)-pd.to_datetime(time2)
    diff = float(abs(diff.days.values[0]))
    return diff


X = df[['Origin Time']].values
 


from sklearn.cluster import DBSCAN
import numpy as np
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import pdist


maxcluster = aa.labels_.max() + 1
mincluster = aa.labels_.min()

counttotal = 0

for i in range(mincluster , maxcluster):
    
    
    
    dftemp = df[df['cluster_label']==i]

    
    if i < 0:
        
        if len(dftemp) > 0:
            
            earth_quake_without_sequence += len(dftemp)
        continue
        
        
    if len(dftemp) <=1:
        if len(dftemp) == 1:
            earth_quake_without_sequence += 1
        
        continue
    
    X = dftemp[['Origin Time']].values    
    
    aa2 = DBSCAN( eps=7,min_samples=1,metric=cluster_distance_time)
    aa2 = aa2.fit(X)

    
    dftemp[ 'time_cluster'] = aa2.labels_
    
    tempmax = aa2.labels_.max()+1
    tempmin = aa2.labels_.min()

    
    
    for l  in range (tempmin, tempmax):
        
        
        dftemplen = len(dftemp[dftemp[ 'time_cluster'] == l])
        
        counttotal += dftemplen
        
        if l < 0:
            if dftemplen > 0:
            
                earth_quake_without_sequence += dftemplen
   
            continue
    
        if dftemplen > 1:
            earth_quake_with_sequence += dftemplen
        elif dftemplen == 1:
            earth_quake_without_sequence +=1
    
    

In [ ]:
print("EARTHQUAKE WITHOUT SEQUENCE  = ", earth_quake_without_sequence)
print("EARTHQUAKE WITH SEQUENCE  = ", earth_quake_with_sequence)
# earth_quake_with_sequence + earth_quake_without_sequence

In [ ]:
print("TOTAL EARTHQUAKE COUNT CHECK \n[earth_quake_with_sequence + earth_quake_without_sequence = len(df)] ? = ",(earth_quake_with_sequence + earth_quake_without_sequence) == len(df))

In [ ]:
print("PROBABILITY OF SEQUENCE EARTHQUAKE = ",(earth_quake_with_sequence/len(df))*100 ,"%")

## CONCLUTION
We can see that ,there is a probability of 11.6% of occurring a sequence of earthquake within short distance and time. 


## REQUEST 

#### I don't know the right way to do this .Just tried the resources I could find. Requesting to comment resources ,if there is any good way to do this .